In [1]:
        // Формируем массив корреляционных зависимостей данной колонки с другими
        // остальными колонками. С колонкой связан массив mapDictCorr. Его элементы
        // определяют набор текстов(слов), которые встречаются в колонке. Берем колонку,
        // переписываем все слова-тексты, которые встречаются в колонке, и формируем из
        // них элементы массива mapDictCorr. Что представляют элементы массива mapDictCorr?
        // Это списки с именами слов-текстов рассматриваемой колонки. Каждый слово-текст
        // колонки, в свою очередь, коммуницирует с текстами других колонок. Поэтому текст
        // тоже должен быть представлен списком(массивом). Этот список состоящих из пар.
        // В паре первое слово ключевое и будет определять слово-текст соседней колонки.
        // Второй элемент - это количество раз, когда данное слово  встречалось в колонке
        // вместе ключевым рассматриваемым словом.
        


@file:Repository("https://repo1.maven.org/maven2")
@file:DependsOn("org.nield:kotlin-statistics:1.2.1")

In [2]:
%use numpy 

In [3]:
%use lets-plot, krangl

In [4]:
val sectors = DataFrame.readCSV("/home/python/python/DataSet/data/sectors.csv")
val sector_categories = DataFrame.readCSV("/home/python/python/DataSet/data/sector_categories.csv")

In [5]:
sectors.head(3)

category code voluntary_code name_en description_en name_fr description_fr 111 11110 Education policy and administrative management Education sector policy, planning and programme... Politique de l’éducation et gestion administrative Politique de l’éducation, planification et prog... 111 11120 Education facilities and training Educational buildings, equipment, materials; su... Equipements scolaires et formation Bâtiments scolaires, équipement, fournitures ; ... 111 11130 Teacher training Teacher education (where the level of education... Formation des enseignants Éducation des enseignants (quand le niveau d’éd...

In [6]:
sector_categories.head(3)

code,name_en,name_fr
110,Education,Education
111,"Education, Level Unspecified","Education, Niveau non spécifié"
112,Basic Education,Education de Base


Два DataFrame'ы объединяем по общему полю. Наименование общего поля формируем из существующих полей путем их переименования sector_categories.rename("code" to "code1") и sectors.rename("category" to "code1"). Далее указываем это поле как ключ объединения by = "code1". Наименования полей двух DataFrame'ов могут совпадать. Для их различения используют суфиксы suffices = "_1" to "_2". Наименованиие совпадающих полей первого DataFrame'а будет оканчиваться на _1, а второго - _2.

In [7]:
val df = sector_categories.rename("code" to "code1").innerJoin(sectors.rename("category" to "code1"), by = "code1", suffices = "_1" to "_2")
df.head(3)

code1 name_en_1 name_fr_1 code voluntary_code name_en_2 description_en name_fr_2 description_fr 111 Education, Level Unspecified Education, Niveau non spécifié 11110 Education policy and administrative management Education sector policy, planning and programme... Politique de l’éducation et gestion administrative Politique de l’éducation, planification et prog... 111 Education, Level Unspecified Education, Niveau non spécifié 11120 Education facilities and training Educational buildings, equipment, materials; su... Equipements scolaires et formation Bâtiments scolaires, équipement, fournitures ; ... 111 Education, Level Unspecified Education, Niveau non spécifié 11130 Teacher training Teacher education (where the level of education... Formation des enseignants Éducation des enseignants (quand le niveau d’éd...

In [26]:
//////////////////////////////////////////////////////////
//Класс Column держит всю информацию о колонке DataFrame//
//////////////////////////////////////////////////////////


class Column(val idCol: Int, val name: String) {

         
    // словарь индексов данного слова в множестве setsDict 
    var mapIndexSets: MutableMap<String, Int>? = mutableMapOf()           
    
    
    // по Int выбираю колонку, по String выбираю Множество-СловоКолонкиКласса
    var mapDictCorr: MutableMap<Int, MutableMap<String, Triple<Int, String, MutableSet<String>?>>?> = mutableMapOf()
    
      
    var indexColClass: Int = 0
    var nameColClass:  String = ""
    
    // каждая колонка при инициализации получит свой индекс и свое имя
    init {
        indexColClass = idCol
        nameColClass  = name
    }
    
    
    fun update(arrCol: Array<Pair<String, Any?>>) {
    
        println("\n\nHello, $name!")
        print("\n!!!!! countRow = " + countRow)
        
//        arrCol.forEachIndexed { index, element ->     print("\n$index arrCol.first = " 
  //                     + element.first + "   arrCol.second =  " + element.second)}
//!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  
        
  
        
//        val indexColClass = indexColClass
  //      val nameCol  = arrCol[indexColClass].first
        // запись, которая находится в колонке класса  
        val textColClass = arrCol[indexColClass].second.toString()
        
        
//println("\nindexColClass = $indexColClass")
  //println("nameColClass  = $nameColClass ")
    //println("textColClass  = $textColClass ")
        
          
   
        
//        num11?.javaClass?.getMethods()?.forEach { it ->
  //                println("\n @@@@@@@@@@@@ num11 = " + it )  }
  
                  
        // организуем цикл по Row          
        arrCol.forEach{element ->
        
            val indxColRow  = element.first.toString().padStart(3, '0').toInt()
            val textColRow  = element.second.toString()
//print("\n!!!@@@### textColRow = " + textColRow + "    indxColRow = " + indxColRow)
            
            // Cмотрим алфавит множества. Каждая запись колонки класса 
            // будет представлять множество. Достаю индекс по textColClass
            val indxMap1: Int = mapIndexSets?.get(textColClass) ?: -1
//print("\n!!!@@@### indxMap1 = " + indxMap1)
            // Множество колонки класса представлено
            if(indxMap1 > 0) {
                //выбираем МножествоСловоКолонкиКласса по индексу
                val map1 = mapDictCorr.get(indxMap1)
//println("\n!!!@@@### map1 --> " +  map1)
                
                // каждый элемент map1 - тройка из слова колонки Row != Class
                // достаем тройку (НомерКолонки:СловоКолонки:КоличествоСлов)
                var triple1:Triple<Int, String, MutableSet<String>?>? = map1?.get(textColRow)
//print("\n!!!@@@### triple1 = " + triple1 )

                // такого СловаКолонкиСтроки еще не было в МножествеСловоКолонкиКласса 
                if(triple1 == null) {
                    val set1 = mutableSetOf("$countRow")
                    val triple12 = Triple(indxColRow, textColRow, set1) // 1 - слово встретилось первый раз
////print("\n!!!@@@###2 triple12 = " + triple12.first + "   " + triple12.second + "   " + triple12.third)
            
                    // связали слово колонки класса с очередной записью множества
                    val map12: Map<String, Triple<Int, String, MutableSet<String>?>> = mutableMapOf(Pair(textColRow, triple12))
////print("\n!!!@@@###2 map1 = " + map1)

                    map1?.putAll(map12) //добавляем к существующей коллекции новую коллекцию

                    mapDictCorr.put(indxMap1, map1) // новое множество перезаписываем в mapDictCorr
                    
            
//print("\n!!!@@@###2 mapIndexSets --> " + mapIndexSets)
  //print("\n!!!@@@###2 mapDictCorr --> " + mapDictCorr)
                // СловоКолонкиСтроки в МножествеКолонкиКласса представлено    
                } else {       
                      // извлекаем множество
                      var set2 = triple1.third
                      // включаем номер текущей строки в множество
                      set2?.addAll(setOf("$countRow"))
                      // далее по плану
                      val triple2 = Triple(indxColRow, textColRow, set2)
                     
                      map1?.put(textColRow,triple2)
                      mapDictCorr.put(indxMap1, map1)
                }
                
            // Нет множества textColClass - формируем его. Включаем в Алфавит множества Слово Колонки.
            } else {
                // первая запись множества
                val set1 = mutableSetOf("$countRow") 
                val triple1 = Triple(indxColRow, textColRow, set1)
////print("\n!!!@@@###1 triple1 = " + triple1.first + "   " + triple1.second + "   " + triple1.third)
            
                // связали слово колонки класса с первой запись множества
                //                           НомерКолонки ТекстКолонки МножествоСтрок"$countRow"
                val map1: MutableMap<String, Triple<Int, String, MutableSet<String>?>> = mutableMapOf(Pair(textColRow, triple1))
////print("\n!!!@@@###1 map1 = " + map1)
                
                
                // это следующий индекс в mapIndexSets
                val nextMapIndexSets = (mapIndexSets?.size ?: 0) + 1
            
                // сформировали множество textColClass с первой буквой Алфавита textColRow
                mapIndexSets?.put(textColClass, nextMapIndexSets) // nextMapIndexSets - индекс очередного элемента множества
                //                      |---------------|
                mapDictCorr.put(nextMapIndexSets, map1)            
//print("\n!!!@@@###1 mapIndexSets --> " + mapIndexSets)
////print("\n!!!@@@###1 mapDictCorr  --> " + mapDictCorr)

            } // end if() else
        } // end arrCol.forEach         
        
    } // end update
    
    
    
    
    
     fun printIndexColClass() {
        print ("\n ///// var indexColClass = $indexColClass ")
    }
    
    
    fun printNameColClass() {
        print ("\n ///// var nameColClass = $nameColClass ")
    }
    
    
    // словарь индексов данного слова в множестве setsDict
    fun printMapIndexSets(){
        print("\n ///// var mapIndexSets -->  $mapIndexSets " ) 
    }
             
    
    // по Int выбираю колонку, по String выбираю Множество-Слово Колонки
    fun printMapDictCorr() {
        print("\n ///// var mapDictCorr --> $mapDictCorr ")
    }
    
    
    fun print12(){
        val map13 = mapDictCorr.get(1)
                  
        // номера колонк DataFrame, с которыми взаимодействовало слово с mapIndexSets = 1 
        println("\n Номера колонк DataFrame, с которыми взаимодействовало слово с mapIndexSets = 1\n")
        map13?.forEach { it -> print(
        "\n Номера колонк DataFrame, с которыми взаимодействовало слово с mapIndexSets = 1\n"
        + it.value.first 
        + "\n Тексты колонк DataFrame, номера которых приведены выше, для рассматриваемой строки '$countRow'\n"
        + it.value.second
        + "\n Множество номеров строк, где тексты, приведенные выше, встречалися\n"
        + it.value.third) }
        println("\n")
        
        // тексты колонк DataFrame, номера которых приведены выше, для данной строки "$countRow"
        //println("\n Тексты колонк DataFrame, номера которых приведены выше, для рассматриваемой строки '$countRow'\n")
        //map13?.forEach { it -> print("\t " + it.value.second)}
        //println("\n")
        
        // множество номеров строк, где тексты, приведенные выше, встречалися
        //println("\n Множество номеров строк, где тексты, приведенные выше, встречалися\n")
        //map13?.forEach { it -> print("\t " + it.value.third)}
    
        println("\n!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
    }
    
    
    
     // companion object реализует счетчик строк DF,
     // который будет использоваться при реализации скользящего окна)
    companion object{
        var countRow: Long
        
        init { countRow = 0 }
        
        fun counterRow(){ countRow++ }
    }    


}





In [27]:

//////////////////////////////////////////////////////////
//    Подготовка входного массива и его обработка       //
//////////////////////////////////////////////////////////



// составляем список имен колонок DataFrame df
val nameList = listOf(df.names)
// каждая колонка DF будет обрабатываться своим экземплярои класса Column;
// объявляем массив List для myCol[i] для обработки i-ой колонки  
var myCol = ArrayList<Column>(nameList[0].size)

// инициализируем List myCol[i] 
//for(i in 0..nameList[0].size-1) {
for(i in 0 until nameList[0].size) {
    // просто добавляем новый Column() к myCol
    myCol.add(Column(i, nameList[0][i]))
}

/////////////////////////////////////////////////////
// инициализация myCol - нет "0" элемента через forEachIndexed - появился ???
println("nameList[0].size = " + nameList[0].size)
myCol.forEach{it -> print("\nindexColClass = " + it.indexColClass)}
println()
myCol.forEachIndexed { index, element -> println("index = $index  AND  element = " 
            +  element.indexColClass + "   " + element.nameColClass)}
/////////////////////////////////////////////////////




// обнуляем счетчик строк DF (companion object в классе Column)
Column.countRow = 0

// подгтовка закончена; начинаем обработку DF
// бежим по строкам DataFrame
df.rows.forEach { it ->
    // готовим массив из колонок строки для обработки 
    var arrRow = emptyArray<Pair<String,Any?>>()
    
//println("\nvar arrRow.size = " + arrRow.size) //сейчас это нуль
    
    Column.counterRow() //увеличииваем счетчик строк на единицу
    // бежим по колонкам строки и формируем массив arrRow
    for(idCol in 0 until nameList[0].size){
        // строим пару из ID колонки и ее значения
        //var entry = Pair(nameList[0][idCol].toString(), it.get(nameList[0][idCol]))
        var entry = Pair(idCol.toString(), it.get(nameList[0][idCol]))
        // записывем пару для каждой колонки в массив строки 
        arrRow += entry
        
        // служит для определения методов класса (в jupiter это сделать непросто)       
//        it1.javaClass.getMethods().forEach {
  //          println("\n @@@@@@@@@@@@ it1 = " + it)
    //    }
      //  служит для определения класса тестируемой переменной "it"
        //println("\n!!!!!!!!!! it::class.simpleName = " + it1::class.simpleName)
            
    }
    
    
    
    // бежим по колонкам строки и делаем update myCol[idCol]
    for(idCol in 0 until nameList[0].size){
        // делаем update(обработку) колонок выбранной строки
        // idCol уже внесено в myCol[idCol] (поэтому update) при инициализации в блоке init{}
        myCol[idCol].update(arrRow)
        
        myCol[idCol].printIndexColClass()
        
        myCol[idCol].printNameColClass()
        
//        myCol[idCol].printMapIndexSets()
        
  //      myCol[idCol].printMapDictCorr()
        
        myCol[idCol].print12()
    }




//    arrRow.forEach{ it -> print("\narrRow.first = " + it.first + "\narrRow.second =  " + it.second)}

//    arrRow.forEachIndexed { index, element ->     print("\n$index arrRow.first = " 
  //                     + element.first + "   arrRow.second =  " + element.second)}
    
}






nameList[0].size = 9

indexColClass = 0
indexColClass = 1
indexColClass = 2
indexColClass = 3
indexColClass = 4
indexColClass = 5
indexColClass = 6
indexColClass = 7
indexColClass = 8
index = 0  AND  element = 0   code1
index = 1  AND  element = 1   name_en_1
index = 2  AND  element = 2   name_fr_1
index = 3  AND  element = 3   code
index = 4  AND  element = 4   voluntary_code
index = 5  AND  element = 5   name_en_2
index = 6  AND  element = 6   description_en
index = 7  AND  element = 7   name_fr_2
index = 8  AND  element = 8   description_fr


Hello, code1!

!!!!! countRow = 1
 ///// var indexColClass = 0 
 ///// var nameColClass = code1 
 Номера колонк DataFrame, с которыми взаимодействовало слово с mapIndexSets = 1


 Номера колонк DataFrame, с которыми взаимодействовало слово с mapIndexSets = 1
0
 Тексты колонк DataFrame, номера которых приведены выше, для рассматриваемой строки '1'
111
 Множество номеров строк, где тексты, приведенные выше, встречалися
[1]
 Номера колонк DataFram

4
 Тексты колонк DataFrame, номера которых приведены выше, для рассматриваемой строки '1'

 Множество номеров строк, где тексты, приведенные выше, встречалися
[1]
 Номера колонк DataFrame, с которыми взаимодействовало слово с mapIndexSets = 1
5
 Тексты колонк DataFrame, номера которых приведены выше, для рассматриваемой строки '1'
Education policy and administrative management
 Множество номеров строк, где тексты, приведенные выше, встречалися
[1]
 Номера колонк DataFrame, с которыми взаимодействовало слово с mapIndexSets = 1
6
 Тексты колонк DataFrame, номера которых приведены выше, для рассматриваемой строки '1'
Education sector policy, planning and programmes; aid to education ministries, administration and management systems; institution capacity building and advice; school management and governance; curriculum and materials development; unspecified education activities.
 Множество номеров строк, где тексты, приведенные выше, встречалися
[1]
 Номера колонк DataFrame, с которыми вза

[1]


!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!


Hello, name_en_2!

!!!!! countRow = 1
 ///// var indexColClass = 5 
 ///// var nameColClass = name_en_2 
 Номера колонк DataFrame, с которыми взаимодействовало слово с mapIndexSets = 1


 Номера колонк DataFrame, с которыми взаимодействовало слово с mapIndexSets = 1
0
 Тексты колонк DataFrame, номера которых приведены выше, для рассматриваемой строки '1'
111
 Множество номеров строк, где тексты, приведенные выше, встречалися
[1]
 Номера колонк DataFrame, с которыми взаимодействовало слово с mapIndexSets = 1
1
 Тексты колонк DataFrame, номера которых приведены выше, для рассматриваемой строки '1'
Education, Level Unspecified
 Множество номеров строк, где тексты, приведенные выше, встречалися
[1]
 Номера колонк DataFrame, с которыми взаимодействовало слово с mapIndexSets = 1
2
 Тексты колонк DataFrame, номера которых приведены выше, для рассматриваемой строки '1'
Education, Niveau non spécifié
 Множество номеров строк, где тексты, приведенные

[1]
 Номера колонк DataFrame, с которыми взаимодействовало слово с mapIndexSets = 1
6
 Тексты колонк DataFrame, номера которых приведены выше, для рассматриваемой строки '1'
Education sector policy, planning and programmes; aid to education ministries, administration and management systems; institution capacity building and advice; school management and governance; curriculum and materials development; unspecified education activities.
 Множество номеров строк, где тексты, приведенные выше, встречалися
[1]
 Номера колонк DataFrame, с которыми взаимодействовало слово с mapIndexSets = 1
7
 Тексты колонк DataFrame, номера которых приведены выше, для рассматриваемой строки '1'
Politique de l’éducation et gestion administrative
 Множество номеров строк, где тексты, приведенные выше, встречалися
[1]
 Номера колонк DataFrame, с которыми взаимодействовало слово с mapIndexSets = 1
8
 Тексты колонк DataFrame, номера которых приведены выше, для рассматриваемой строки '1'
Politique de l’éducation,

Education facilities and training
 Множество номеров строк, где тексты, приведенные выше, встречалися
[2]
 Номера колонк DataFrame, с которыми взаимодействовало слово с mapIndexSets = 1
6
 Тексты колонк DataFrame, номера которых приведены выше, для рассматриваемой строки '2'
Educational buildings, equipment, materials; subsidiary services to education (boarding facilities, staff housing); language training; colloquia, seminars, lectures, etc.
 Множество номеров строк, где тексты, приведенные выше, встречалися
[2]
 Номера колонк DataFrame, с которыми взаимодействовало слово с mapIndexSets = 1
7
 Тексты колонк DataFrame, номера которых приведены выше, для рассматриваемой строки '2'
Equipements scolaires et formation
 Множество номеров строк, где тексты, приведенные выше, встречалися
[2]
 Номера колонк DataFrame, с которыми взаимодействовало слово с mapIndexSets = 1
8
 Тексты колонк DataFrame, номера которых приведены выше, для рассматриваемой строки '2'
Bâtiments scolaires, équipement, f

 Множество номеров строк, где тексты, приведенные выше, встречалися
[1]
 Номера колонк DataFrame, с которыми взаимодействовало слово с mapIndexSets = 1
7
 Тексты колонк DataFrame, номера которых приведены выше, для рассматриваемой строки '2'
Politique de l’éducation et gestion administrative
 Множество номеров строк, где тексты, приведенные выше, встречалися
[1]
 Номера колонк DataFrame, с которыми взаимодействовало слово с mapIndexSets = 1
8
 Тексты колонк DataFrame, номера которых приведены выше, для рассматриваемой строки '2'
Politique de l’éducation, planification et programmes ; aide aux ministères de l’éducation, à l’administration et au développement de systèmes de gestion, renforcement des capacités institutionnelles et conseils ; gestion et direction des écoles, développement des programmes d’études et des matériels pédagogiques ; activités d’éducation non spécifiées.
 Множество номеров строк, где тексты, приведенные выше, встречалися
[1]
 Номера колонк DataFrame, с которыми в

7
 Тексты колонк DataFrame, номера которых приведены выше, для рассматриваемой строки '2'
Politique de l’éducation et gestion administrative
 Множество номеров строк, где тексты, приведенные выше, встречалися
[1]
 Номера колонк DataFrame, с которыми взаимодействовало слово с mapIndexSets = 1
8
 Тексты колонк DataFrame, номера которых приведены выше, для рассматриваемой строки '2'
Politique de l’éducation, planification et programmes ; aide aux ministères de l’éducation, à l’administration et au développement de systèmes de gestion, renforcement des capacités institutionnelles et conseils ; gestion et direction des écoles, développement des programmes d’études et des matériels pédagogiques ; activités d’éducation non spécifiées.
 Множество номеров строк, где тексты, приведенные выше, встречалися
[1]
 Номера колонк DataFrame, с которыми взаимодействовало слово с mapIndexSets = 1
3
 Тексты колонк DataFrame, номера которых приведены выше, для рассматриваемой строки '2'
11120
 Множество ном

 Множество номеров строк, где тексты, приведенные выше, встречалися
[1]
 Номера колонк DataFrame, с которыми взаимодействовало слово с mapIndexSets = 1
8
 Тексты колонк DataFrame, номера которых приведены выше, для рассматриваемой строки '2'
Politique de l’éducation, planification et programmes ; aide aux ministères de l’éducation, à l’administration et au développement de systèmes de gestion, renforcement des capacités institutionnelles et conseils ; gestion et direction des écoles, développement des programmes d’études et des matériels pédagogiques ; activités d’éducation non spécifiées.
 Множество номеров строк, где тексты, приведенные выше, встречалися
[1]


!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!


Hello, name_fr_2!

!!!!! countRow = 2
 ///// var indexColClass = 7 
 ///// var nameColClass = name_fr_2 
 Номера колонк DataFrame, с которыми взаимодействовало слово с mapIndexSets = 1


 Номера колонк DataFrame, с которыми взаимодействовало слово с mapIndexSets = 1
0
 Тексты колонк DataF

11110
 Множество номеров строк, где тексты, приведенные выше, встречалися
[1]
 Номера колонк DataFrame, с которыми взаимодействовало слово с mapIndexSets = 1
4
 Тексты колонк DataFrame, номера которых приведены выше, для рассматриваемой строки '3'

 Множество номеров строк, где тексты, приведенные выше, встречалися
[1, 2, 3]
 Номера колонк DataFrame, с которыми взаимодействовало слово с mapIndexSets = 1
5
 Тексты колонк DataFrame, номера которых приведены выше, для рассматриваемой строки '3'
Education policy and administrative management
 Множество номеров строк, где тексты, приведенные выше, вст��ечалися
[1]
 Номера колонк DataFrame, с которыми взаимодействовало слово с mapIndexSets = 1
6
 Тексты колонк DataFrame, номера которых приведены выше, для рассматриваемой строки '3'
Education sector policy, planning and programmes; aid to education ministries, administration and management systems; institution capacity building and advice; school management and governance; curriculum and mate

In [22]:

/////////////////////////////////////////////////////////
//            Выводим результаты расчетов              //
/////////////////////////////////////////////////////////

// составляем список имен колонок DataFrame df
val nameList = listOf(df.names)

// находим максимальный размер массива для вывода результир. данных
var max: Int = 0
for(i in 0 until nameList[0].size){
    if(myCol[i].mapIndexSets?.size!! > max) max = myCol[i].mapIndexSets?.size!!
}

val row: Int = nameList[0].size
val column   = max + 1
val array = Array(row, {IntArray(column)})
val array2 = Array(row, {IntArray(row)})


// бежим по элементам колонок 
for(indRow in 0 until row){

println("@@@@@@@@@@@@@@@@@@@@@@@  $indRow")
//println("\n myCol[indRow].mapIndexSets  " + myCol[indRow].mapIndexSets)
    
    // бежим по колонкам выбранной строки        |--- это индексы слов МножестваКлассаКолонки
    for(indMap in myCol[indRow].mapIndexSets?.values!!){
        //                                 |<------------------ map1 ------------------->|  
        //var mapDictCorr: MutableMap<Int, MutableMap<String, Triple<Int, String, Int?>>?>
        //                             |-->----->--|
        val map1 = myCol[indRow].mapDictCorr.get(indMap)
//println("\n ################ indMap =  $indMap")        
//println("\n ################ map1 =  " + map1)
        
        // выбрали множество mapDictCorr.get(indMap) и бежим по его элементам
        var sum: Int = 0
        map1?.forEach { it ->          
            // val enteryKey  = it.key
            // достаем тройку  (ИндексКолонки : СловоКолонки : КоличествоСлов)
            val enteryTriple: Triple<Int,          String,    MutableSet<String>?>   =   it.value
           
           
            val indCol2: Int = enteryTriple.first.toInt()
            val sizeSets: Int? = enteryTriple.third?.size
            // допускаем, что корреляционная зависимость существует,
            // если кол-во совпадений больше двух
            if(sizeSets!! > 2)
                array2[indRow][indCol2] +=  sizeSets!!
            
            if(sizeSets!! > 2)
                sum = sum.plus(sizeSets)
        }


        array[indRow][indMap] = sum
    }
}





@@@@@@@@@@@@@@@@@@@@@@@  0
@@@@@@@@@@@@@@@@@@@@@@@  1
@@@@@@@@@@@@@@@@@@@@@@@  2
@@@@@@@@@@@@@@@@@@@@@@@  3
@@@@@@@@@@@@@@@@@@@@@@@  4
@@@@@@@@@@@@@@@@@@@@@@@  5
@@@@@@@@@@@@@@@@@@@@@@@  6
@@@@@@@@@@@@@@@@@@@@@@@  7
@@@@@@@@@@@@@@@@@@@@@@@  8


In [23]:
// бежим по строкам
for(indRow in 0 until row){
    println()
    println()
    // бежим по колонкам выбранной строки
    for(indCol in 0 until column){
        print ("\t " + array[indRow][indCol]) 
    }
}

println()

// бежим по строкам
for(indRow in 0 until row){
    println()
    println()
    // бежим по колонкам выбранной строки
    for(indCol in 0 until row){
        print ("\t " + array2[indRow][indCol]) 
    }
}

println()






	 0	 16	 25	 15	 0	 19	 32	 24	 23	 44	 175	 24	 76	 58	 23	 26	 36	 24	 0	 0	 28	 24	 16	 54	 24	 20	 76	 40	 0	 24	 0	 24	 58	 0	 0	 0	 28	 21	 0	 0	 0	 36	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0	 0

	 0	 16	 25	 15	 0	 19	 32	 24	 23	 44	 175	 24	 76	 58	 23	 26	 36	 24	 0	 0	 2

	 212	 190	 212	 0	 230	 0	 0	 0	 0

	 0	 0	 0	 0	 0	 0	 0	 0	 0

	 5	 5	 5	 0	 0	 0	 0	 0	 24

	 0	 0	 0	 0	 0	 0	 0	 0	 0

	 9	 9	 9	 0	 0	 0	 0	 0	 29


In [12]:
//val map = mutableMapOf("One" to 1, "Two" to 2, "Three" to 3)
//map.put("Four", 4)
//map += "Five" to 5
//map += "Two" to 22
//map.put("Two", 2)
//println(map) //{One=1, Two=22, Three=3, Four=4, Five=5}




lateinit var mapFromSet: MutableMap<String, Set<String>> //? = mutableMapOf("lst1" to setOf("1", "3", "7", "9"))
mapFromSet = mutableMapOf("lst1" to setOf("1", "3", "7", "9"))

var lst = setOf("1", "3", "7", "9", "6")

mapFromSet?.plusAssign(Pair("lst1",  lst))
mapFromSet?.plusAssign(Pair("lst2",  setOf("2", "4")))
mapFromSet?.plusAssign(Pair("lst3",  setOf("5", "8")))

println()
//println(lst)
println(mapFromSet) //{lst1=[1, 3, 7, 9, 6], lst2=[2, 4], lst3=[5, 8]}


//val set1: Set<String>> = mapFromSet?.get("lst1")

var set2 = mapFromSet.get("lst1") as MutableSet
set2.addAll(setOf("34"))
mapFromSet.remove("lst1")
mapFromSet.plusAssign(Pair("lst1", set2))

println(mapFromSet) //{lst1=[1, 3, 7, 9, 6], lst2=[2, 4], lst3=[5, 8]}




{lst1=[1, 3, 7, 9, 6], lst2=[2, 4], lst3=[5, 8]}
{lst2=[2, 4], lst3=[5, 8], lst1=[1, 3, 7, 9, 6, 34]}
